<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n20_x6_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n20_x6_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    16.093894  47.133563  ... -2.066951e+08  1.903497e+08
     1.0    11.031800  39.130969  ...  5.265155e+07 -5.417166e+07
     2.0    31.692152  18.424195  ...  2.425335e+08  3.841991e+08
     3.0    32.105397  37.606530  ... -1.824779e+08  6.543978e+08
     4.0    24.566404  23.020840  ... -3.265428e+07  7.315190e+07
...               ...        ...  ...           ...           ...
99.0 15.0   18.864664  13.161906  ... -7.136487e+06  2.943471e+07
     16.0   38.484045  27.417162  ...  1.016064e+08 -3.120730e+09
     17.0   29.450201  43.053435  ... -6.089170e+07  1.303128e+09
     18.0   44.029514  26.800473  ...  1.664266e+08  3.374722e+09
     19.0   24.044657  26.331132  ...  2.184550e+08 -4.900360e+08

[2000 rows x 7 columns]

In [3]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    16.093894  47.133563  ... -2.066951e+08  1.903497e+08
     1.0    11.031800  39.130969  ...  5.265155e+07 -5.417166e+07
     2.0    31.692152  18.424195  ...  2.425335e+08  3.841991e+08
     3.0    32.105397  37.606530  ... -1.824779e+08  6.543978e+08
     4.0    24.566404  23.020840  ... -3.265428e+07  7.315190e+07
...               ...        ...  ...           ...           ...
99.0 15.0   18.864664  13.161906  ... -7.136487e+06  2.943471e+07
     16.0   38.484045  27.417162  ...  1.016064e+08 -3.120730e+09
     17.0   29.450201  43.053435  ... -6.089170e+07  1.303128e+09
     18.0   44.029514  26.800473  ...  1.664266e+08  3.374722e+09
     19.0   24.044657  26.331132  ...  2.184550e+08 -4.900360e+08

[2000 rows x 7 columns]

In [4]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,16.093894,47.133563,39.915556,-1.634564e+07,-1.634539e+07,-2.066951e+08,1.903497e+08
1,0.0,1.0,11.031800,39.130969,21.786820,-1.520279e+06,-1.520102e+06,5.265155e+07,-5.417166e+07
2,0.0,2.0,31.692152,18.424195,10.086199,6.267322e+08,6.267326e+08,2.425335e+08,3.841991e+08
3,0.0,3.0,32.105397,37.606530,38.714802,4.719195e+08,4.719199e+08,-1.824779e+08,6.543978e+08
4,0.0,4.0,24.566404,23.020840,33.718253,4.049731e+07,4.049762e+07,-3.265428e+07,7.315190e+07
...,...,...,...,...,...,...,...,...,...
1995,99.0,15.0,18.864664,13.161906,37.132320,2.229798e+07,2.229822e+07,-7.136487e+06,2.943471e+07
1996,99.0,16.0,38.484045,27.417162,16.985660,-3.019124e+09,-3.019124e+09,1.016064e+08,-3.120730e+09
1997,99.0,17.0,29.450201,43.053435,26.117377,1.242236e+09,1.242236e+09,-6.089170e+07,1.303128e+09
1998,99.0,18.0,44.029514,26.800473,10.392964,3.541148e+09,3.541148e+09,1.664266e+08,3.374722e+09


In [5]:
table.set_index(['rep', 'index'], inplace=True)

In [6]:
table

x1         x2  ...         y_hat         e_hat
rep  index                        ...                            
0.0  0.0    16.093894  47.133563  ... -2.066951e+08  1.903497e+08
     1.0    11.031800  39.130969  ...  5.265155e+07 -5.417166e+07
     2.0    31.692152  18.424195  ...  2.425335e+08  3.841991e+08
     3.0    32.105397  37.606530  ... -1.824779e+08  6.543978e+08
     4.0    24.566404  23.020840  ... -3.265428e+07  7.315190e+07
...               ...        ...  ...           ...           ...
99.0 15.0   18.864664  13.161906  ... -7.136487e+06  2.943471e+07
     16.0   38.484045  27.417162  ...  1.016064e+08 -3.120730e+09
     17.0   29.450201  43.053435  ... -6.089170e+07  1.303128e+09
     18.0   44.029514  26.800473  ...  1.664266e+08  3.374722e+09
     19.0   24.044657  26.331132  ...  2.184550e+08 -4.900360e+08

[2000 rows x 7 columns]

In [8]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid
    SSR1 = model1.ssr
    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid
    SSR0 = model0.ssr
    MSE0 = model0.mse_resid

    df1 = DF1+1
    df0 = DF0+1

    # คำนวณค่าสถิติ
    GQ_test_clusters = (SSR1/df1) / (SSR0/df0)


    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

In [9]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,6.0,8.0,1.720908649292609e+18,1.618110612190868e+21,3.441817298585218e+17,2.3115865888440972e+20,705.1989422227625,4.785038210108894e-08,0.999999976074809,Reject005=0 : Heteroscedasticity
2.0,6.0,8.0,9590585408197696.0,7.45318059669483e+20,1918117081639539.2,1.0647400852421186e+20,58285.13286314185,8.526512829121202e-14,0.9999999999999574,Reject005=0 : Heteroscedasticity
3.0,9.0,5.0,2.268307187800465e+19,1.1991639503669563e+19,2.835383984750581e+18,2.997909875917391e+18,0.9515885336295979,0.9872527581374153,0.5063736209312923,Reject005=1 : Homoscedasticity
4.0,7.0,7.0,1.622992711047273e+20,3.03846683392237e+20,2.704987851745455e+19,5.0641113898706166e+19,1.8721383116759227,0.42698162253253713,0.7865091887337314,Reject005=1 : Homoscedasticity
7.0,10.0,4.0,6.911479786858291e+18,2.656284630421669e+18,7.679421985398102e+17,8.854282101405563e+17,0.960823409869625,0.9394920935880215,0.5302539532059892,Reject005=1 : Homoscedasticity
8.0,5.0,9.0,1.4920838602252163e+20,1.2894070818040992e+21,3.730209650563041e+19,1.611758852255124e+20,4.8009182778829835,0.0988221604425994,0.9505889197787003,Reject005=1 : Homoscedasticity
9.0,10.0,4.0,1.0647434939548746e+17,3.6611546363382104e+20,1.1830483266165272e+16,1.2203848787794035e+20,8596.33014224686,2.220446049250313e-16,0.9999999999999999,Reject005=0 : Heteroscedasticity
10.0,9.0,5.0,4.515914234398553e+17,3.7939622325877087e+20,5.644892792998191e+16,9.484905581469272e+19,1512.2368725781184,1.4235279621743757e-12,0.9999999999992882,Reject005=0 : Heteroscedasticity
15.0,7.0,7.0,1.5147993847403565e+19,1.299176065739406e+20,2.5246656412339277e+18,2.16529344289901e+19,8.576555277397942,0.011098733492863833,0.9944506332535681,Reject005=0 : Heteroscedasticity


In [10]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,9.0,5.0,2.2837160165738345e+20,1.1532596751535034e+16,2.854645020717293e+19,2883149187883758.5,9.089866691878113e-05,6.749085018893554e-10,Reject005=0 : Heteroscedasticity
1.0,6.0,8.0,1.720908649292609e+18,1.618110612190868e+21,3.441817298585218e+17,2.3115865888440972e+20,705.1989422227625,4.785038210108894e-08,Reject005=0 : Heteroscedasticity
2.0,6.0,8.0,9590585408197696.0,7.45318059669483e+20,1918117081639539.2,1.0647400852421186e+20,58285.13286314185,8.526512829121202e-14,Reject005=0 : Heteroscedasticity
3.0,9.0,5.0,2.268307187800465e+19,1.1991639503669563e+19,2.835383984750581e+18,2.997909875917391e+18,0.9515885336295979,1.0127472418625847,Reject005=1 : Homoscedasticity
4.0,7.0,7.0,1.622992711047273e+20,3.03846683392237e+20,2.704987851745455e+19,5.0641113898706166e+19,1.8721383116759227,0.42698162253253713,Reject005=1 : Homoscedasticity
5.0,8.0,6.0,1.341853920734713e+20,6.600117718693089e+17,1.9169341724781613e+19,1.320023543738618e+17,0.0065582078811075445,4.63891237359681e-06,Reject005=0 : Heteroscedasticity
6.0,9.0,5.0,2.794794177317606e+20,5.57048906436201e+19,3.4934927216470073e+19,1.3926222660905026e+19,0.35876990145568577,0.2714462764489295,Reject005=1 : Homoscedasticity
7.0,10.0,4.0,6.911479786858291e+18,2.656284630421669e+18,7.679421985398102e+17,8.854282101405563e+17,0.960823409869625,1.0605079064119785,Reject005=1 : Homoscedasticity
8.0,5.0,9.0,1.4920838602252163e+20,1.2894070818040992e+21,3.730209650563041e+19,1.611758852255124e+20,4.8009182778829835,0.0988221604425994,Reject005=1 : Homoscedasticity


In [11]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=0 : Heteroscedasticity    74
Reject005=1 : Homoscedasticity      26
Name: Result_test, dtype: int64